In [ ]:
# Author: Sam Joshua Caballero
# Date: 2025-06-13
# Description: This notebook automates the calculations done to generate SEO website performance reports of Jem Enterprise.

In [5]:
import pandas as pd
import os

## User Inputs

In [6]:
month = "Insert month here"
gsc_file = "GSC Input/queries.csv"
ahrefs_keywords = "AHREFS Input/Ahrefs Keyword File"
ahrefs_pages = "AHREFS Input/Ahrefs Top Pages File"
save_csv_files = True

# Process 1 A Configurations
top_ten = True
top_twenty = False
top_one_hundred = False
queries_with_most_clicks = 10 # options: 5, 10, 15, and 20
queries_with_most_impressions = 15 # options: 10, 15, 20

# Process 1 B Configurations 
top_5_best_performing_pages = False
top_5_worst_performing_pages = False

# Process 2 B Configurations 
top_5_best_performing_pages = False
top_5_worst_performing_pages = False

In [7]:
# === Main Process ===
try:
    df = pd.read_csv(gsc_file)
except FileNotFoundError:
    print(f"Error: File '{gsc_file}' not found. Please check the file name or path.")
    exit()
except Exception as e:
    print(f"Error reading file: {e}")
    exit()

## Process 1: Google Search Console

### Process 1 (Output 1): Total Number of Queries

In [8]:
# Process
total_number_of_queries = df.shape[0]

# Print Results
print(f"Total Number of Queries: {total_number_of_queries}")

Total Number of Queries: 393


### Process 1 (Outputs 2-4): Queries with average position 1-10, 11-20, and 21-100

In [9]:
# Process
def categorize_by_position(df, top_ten=False, top_twenty=False, top_one_hundred=False):
    result = {}

    if top_ten:
        result['top_1_to_10'] = df[(df['Position'] >= 1) & (df['Position'] <= 10)]

    if top_twenty:
        result['pos_11_to_20'] = df[(df['Position'] > 10) & (df['Position'] <= 20)]

    if top_one_hundred:
        result['pos_21_to_100'] = df[(df['Position'] > 20) & (df['Position'] <= 100)]

    return result
    
categories = categorize_by_position(df, top_ten=True, top_twenty=True, top_one_hundred=True)

# Print Results
for label, data in categories.items():
    print(f"\nQueries/Keywords with Position Range: {label.replace('_', ' ').title()}")
    print(data.shape[0])


Queries/Keywords with Position Range: Top 1 To 10
2

Queries/Keywords with Position Range: Pos 11 To 20
11

Queries/Keywords with Position Range: Pos 21 To 100
379


### Process 1 (Outputs 5): Top x queries with the most clicks

In [10]:
# Process
def get_top_queries_by_clicks(df, queries_with_most_clicks=10):
    top_clicks_df = df.sort_values(by='Clicks', ascending=False).head(queries_with_most_clicks)
    return top_clicks_df

top_clicks = get_top_queries_by_clicks(df, queries_with_most_clicks)
top_clicks = top_clicks.reset_index(drop=True)
top_clicks.insert(0, 'Rank', top_clicks.index + 1)

# Print Results
print(f"\nTop {queries_with_most_clicks} Queries with the Most Clicks:")
print(top_clicks[['Rank', 'Top queries', 'Impressions', 'Clicks', 'CTR', 'Position']])

# Save to CSV
if save_csv_files:
    output_folder = "Output"
    os.makedirs(output_folder, exist_ok=True) 
    top_clicks.to_csv(f'{output_folder}/Top_{queries_with_most_clicks}_Queries_By_Clicks.csv', index=False)
    print(f"Saved to {output_folder}/Top_{queries_with_most_clicks}_Queries_By_Clicks.csv")


Top 10 Queries with the Most Clicks:
   Rank                               Top queries  Impressions  Clicks    CTR  \
0     1            how many lights for a 6ft tree          324       1  0.31%   
1     2                   flocked christmas trees          381       0     0%   
2     3          what is a flocked christmas tree          304       0     0%   
3     4  how many lights for a 6ft christmas tree          176       0     0%   
4     5        how many lights for christmas tree          169       0     0%   
5     6                    what is a flocked tree          162       0     0%   
6     7                     unlit christmas trees          158       0     0%   
7     8    how to decorate a white christmas tree          152       0     0%   
8     9         frosted vs flocked christmas tree          142       0     0%   
9    10                    flocked christmas tree          141       0     0%   

   Position  
0     31.32  
1     58.41  
2     44.37  
3     33.37  


### Process 1 (Outputs 6): Top x queries with the most impressions

In [11]:
# Process
def get_top_queries_by_impressions(df, queries_with_most_impressions=15):
    top_impressions_df = df.sort_values(by='Impressions', ascending=False).head(queries_with_most_impressions)
    return top_impressions_df

top_impressions = get_top_queries_by_impressions(df, queries_with_most_impressions)
top_impressions = top_impressions.reset_index(drop=True)
top_impressions.insert(0, 'Rank', top_impressions.index + 1)

# Print Results
print(f"\nTop {queries_with_most_impressions} Queries with the Most Impressions:")
print(top_impressions[['Rank', 'Top queries', 'Impressions', 'Clicks', 'CTR', 'Position']])

# Save to CSV
if save_csv_files:
    output_folder = "Output"
    os.makedirs(output_folder, exist_ok=True)  
    top_impressions.to_csv(f'{output_folder}/Top_{queries_with_most_impressions}_Queries_By_Impressions.csv', index=False)
    print(f"Saved to {output_folder}/Top_{queries_with_most_impressions}_Queries_By_Impressions.csv")


Top 15 Queries with the Most Impressions:
    Rank                               Top queries  Impressions  Clicks  \
0      1                   flocked christmas trees          381       0   
1      2            how many lights for a 6ft tree          324       1   
2      3          what is a flocked christmas tree          304       0   
3      4  how many lights for a 6ft christmas tree          176       0   
4      5        how many lights for christmas tree          169       0   
5      6                    what is a flocked tree          162       0   
6      7                     unlit christmas trees          158       0   
7      8    how to decorate a white christmas tree          152       0   
8      9         frosted vs flocked christmas tree          142       0   
9     10                    flocked christmas tree          141       0   
10    11                      white christmas tree          134       0   
11    12            what is flocked christmas tree       